In [1]:
import torch
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
from torchvision import datasets, transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder("../data/hymenoptera_data", data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 8, num_workers = 0, shuffle = True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [3]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), 
                loss.item()))

In [4]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = "sum").item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [5]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

model = models.resnet18(pretrained = False).cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print("[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/397 (0%)]	Loss: 7.608947
Train Epoch: 1 [80/397 (20%)]	Loss: 6.886409
Train Epoch: 1 [160/397 (40%)]	Loss: 5.834793
Train Epoch: 1 [240/397 (60%)]	Loss: 3.741014
Train Epoch: 1 [320/397 (80%)]	Loss: 2.774441
[1] Test Loss: 3.1830, accuracy: 48.61%

Train Epoch: 2 [0/397 (0%)]	Loss: 2.669162
Train Epoch: 2 [80/397 (20%)]	Loss: 1.331759
Train Epoch: 2 [160/397 (40%)]	Loss: 0.935329
Train Epoch: 2 [240/397 (60%)]	Loss: 1.068789
Train Epoch: 2 [320/397 (80%)]	Loss: 0.873359
[2] Test Loss: 0.8299, accuracy: 53.65%

Train Epoch: 3 [0/397 (0%)]	Loss: 0.651481
Train Epoch: 3 [80/397 (20%)]	Loss: 0.733070
Train Epoch: 3 [160/397 (40%)]	Loss: 0.628282
Train Epoch: 3 [240/397 (60%)]	Loss: 0.777180
Train Epoch: 3 [320/397 (80%)]	Loss: 0.756756
[3] Test Loss: 0.7500, accuracy: 62.22%

Train Epoch: 4 [0/397 (0%)]	Loss: 0.637236
Train Epoch: 4 [80/397 (20%)]	Loss: 0.576580
Train Epoch: 4 [160/397 (40%)]	Loss: 0.875668
Train Epoch: 4 [240/397 (60%)]	Loss: 0.792054
Train Epoch: 4 [320

In [6]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

if USE_CUDA:
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr = 0.0001)
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print("[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/397 (0%)]	Loss: 0.655521
Train Epoch: 1 [80/397 (20%)]	Loss: 0.673501
Train Epoch: 1 [160/397 (40%)]	Loss: 0.844047
Train Epoch: 1 [240/397 (60%)]	Loss: 0.897998
Train Epoch: 1 [320/397 (80%)]	Loss: 0.821491
[1] Test Loss: 0.5612, accuracy: 71.28%

Train Epoch: 2 [0/397 (0%)]	Loss: 0.647867
Train Epoch: 2 [80/397 (20%)]	Loss: 0.532273
Train Epoch: 2 [160/397 (40%)]	Loss: 0.480060
Train Epoch: 2 [240/397 (60%)]	Loss: 0.744888
Train Epoch: 2 [320/397 (80%)]	Loss: 0.587056
[2] Test Loss: 0.5118, accuracy: 76.07%

Train Epoch: 3 [0/397 (0%)]	Loss: 0.400575
Train Epoch: 3 [80/397 (20%)]	Loss: 0.716245
Train Epoch: 3 [160/397 (40%)]	Loss: 0.323893
Train Epoch: 3 [240/397 (60%)]	Loss: 0.518107
Train Epoch: 3 [320/397 (80%)]	Loss: 0.911845
[3] Test Loss: 0.4280, accuracy: 80.60%

Train Epoch: 4 [0/397 (0%)]	Loss: 0.467954
Train Epoch: 4 [80/397 (20%)]	Loss: 0.450218
Train Epoch: 4 [160/397 (40%)]	Loss: 0.457721
Train Epoch: 4 [240/397 (60%)]	Loss: 0.369129
Train Epoch: 4 [320

In [7]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr = 0.0001)

EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print("[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/397 (0%)]	Loss: 0.899593
Train Epoch: 1 [80/397 (20%)]	Loss: 0.646311
Train Epoch: 1 [160/397 (40%)]	Loss: 0.729588
Train Epoch: 1 [240/397 (60%)]	Loss: 0.449810
Train Epoch: 1 [320/397 (80%)]	Loss: 0.825033
[1] Test Loss: 0.7029, accuracy: 56.42%

Train Epoch: 2 [0/397 (0%)]	Loss: 0.756958
Train Epoch: 2 [80/397 (20%)]	Loss: 0.611335
Train Epoch: 2 [160/397 (40%)]	Loss: 0.525351
Train Epoch: 2 [240/397 (60%)]	Loss: 0.745544
Train Epoch: 2 [320/397 (80%)]	Loss: 0.614920
[2] Test Loss: 0.6996, accuracy: 57.43%

Train Epoch: 3 [0/397 (0%)]	Loss: 0.731268
Train Epoch: 3 [80/397 (20%)]	Loss: 0.665437
Train Epoch: 3 [160/397 (40%)]	Loss: 0.492667
Train Epoch: 3 [240/397 (60%)]	Loss: 0.714585
Train Epoch: 3 [320/397 (80%)]	Loss: 0.774841
[3] Test Loss: 0.6945, accuracy: 56.93%

Train Epoch: 4 [0/397 (0%)]	Loss: 0.530404
Train Epoch: 4 [80/397 (20%)]	Loss: 0.577083
Train Epoch: 4 [160/397 (40%)]	Loss: 0.632086
Train Epoch: 4 [240/397 (60%)]	Loss: 0.639706
Train Epoch: 4 [320